In [3]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.path import Path
from matplotlib.patches import PathPatch

# Load the RTSTRUCT file
# Replace with your actual path
rtstruct_path = "data/SAMPLE_001/RS.1.2.246.352.221.46272062591570509005209218152822185346.dcm"
rtstruct = pydicom.dcmread(rtstruct_path)

# Function to extract contours from RTSTRUCT
def get_contours(rtstruct):
    contours = {}
    
    # Get ROI (Region of Interest) names from the structure set
    roi_names = {roi.ROINumber: roi.ROIName for roi in rtstruct.StructureSetROISequence}
    
    # Extract contour data
    for roi in rtstruct.ROIContourSequence:
        roi_number = roi.ReferencedROINumber
        roi_name = roi_names[roi_number]
        contours[roi_name] = []
        
        # Check if this ROI has contours
        if hasattr(roi, 'ContourSequence'):
            for contour in roi.ContourSequence:
                # Extract contour points
                contour_data = contour.ContourData
                # Convert flat array to points (x,y,z)
                points = np.array(contour_data).reshape(-1, 3)
                contours[roi_name].append(points)
    
    return contours

# Get contours from the RTSTRUCT file
contours = get_contours(rtstruct)
print(f"Available contours: {list(contours.keys())}")

# Function to visualize contours in 2D (for each axial slice)
def visualize_contours_2d(contours_dict):
    # Define colors for each ROI
    colors = {'InitMatchIso': 'red', 'InitLaserIso': 'green', 'AcqIsocenter': 'blue'}
    
    # Find all unique Z positions across all contours
    all_z_positions = []
    for roi_name, roi_contours in contours_dict.items():
        for contour in roi_contours:
            z_positions = np.unique(contour[:, 2])
            all_z_positions.extend(z_positions)
    
    unique_z_positions = np.unique(all_z_positions)
    print(f"Found {len(unique_z_positions)} unique Z positions")
    
    # Create a plot for each unique Z position (slice)
    for z_pos in unique_z_positions:
        plt.figure(figsize=(10, 10))
        plt.title(f"Contours at Z = {z_pos:.2f}")
        
        # Add each ROI's contours that match this Z position
        for roi_name, roi_contours in contours_dict.items():
            # color = colors.get(roi_name, 'gray')
            
            for contour in roi_contours:
                # Get points that match this Z position (with small tolerance)
                mask = np.abs(contour[:, 2] - z_pos) < 0.1
                if np.any(mask):
                    points = contour[mask]
                    x, y = points[:, 0], points[:, 1]
                    plt.plot(x, y, '-', linewidth=2, label=roi_name)
                    
                    # If it's an isocenter marker, it might be just a point
                    if len(x) <= 1:
                        plt.scatter(x, y, s=100, marker='o')
        
        # Add legend without duplicates
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys(), loc='upper right')
        
        plt.axis('equal')
        plt.grid(True)
        plt.savefig(f"img/contour_slice_z{z_pos:.2f}.png")
        plt.close()

# Function to visualize all contours in 3D
def visualize_contours_3d(contours_dict):
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Define colors for each ROI
    colors = {'InitMatchIso': 'red', 'InitLaserIso': 'green', 'AcqIsocenter': 'blue'}
    markers = {'InitMatchIso': 'o', 'InitLaserIso': 's', 'AcqIsocenter': '^'}
    
    # Plot each ROI
    for roi_name, roi_contours in contours_dict.items():
        color = colors.get(roi_name, 'gray')
        marker = markers.get(roi_name, 'o')
        
        for contour in roi_contours:
            x, y, z = contour[:, 0], contour[:, 1], contour[:, 2]
            
            # If it looks like a closed contour (more than 2 points)
            if len(x) > 2:
                ax.plot(x, y, z, '-', color=color, linewidth=2, label=roi_name)
            
            # Plot points for all contours
            ax.scatter(x, y, z, color=color, s=50, marker=marker)
    
    # Add legend without duplicates
    handles, labels = ax.get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    ax.legend(by_label.values(), by_label.keys(), loc='upper right')
    
    ax.set_xlabel('X (mm)')
    ax.set_ylabel('Y (mm)')
    ax.set_zlabel('Z (mm)')
    ax.set_title('3D Visualization of All Contours')
    
    plt.savefig("contours_3d.png")
    plt.show()

# Visualize contours in 2D and 3D
print("Visualizing contours in 2D and 3D...")
visualize_contours_2d(contours)
print("Done")
#visualize_contours_3d(contours)

Available contours: ['zRingPTV_Mid01', 'zRingPTV_Mid00', 'CouchSurface', 'CouchInterior', 'zRingPTV_Low', 'zRingPTV_High', 'zOptPTV_Mid01', 'zOptPTV_Mid00', 'zOptPTV_Low', 'zOptPTV_High', 'z_Planveri', 'z_Avoid', 'z_Artifact', 'Trachea', 'SpinalCord_PRV', 'SpinalCord', 'Retina_R', 'Retina_L', 'PTV_Mid01', 'PTV_Mid00', 'PTV_Low', 'PTV_High', 'PTV_all', 'Pituitary', 'Parotid_R', 'Parotid_L', 'Oral_Cavity-PTV', 'Oral_Cavity', 'OpticNrv_R', 'OpticNrv_L', 'OpticChiasm', 'Musc_Constrict', 'Mandible', 'Lungs', 'Lens_R', 'Lens_L', 'Heart', 'GTV', 'Glottis', 'Glnd_Submand_R', 'Glnd_Submand_L', 'Glnd_Lacrimal_R', 'Glnd_Lacrimal_L', 'Eye_R', 'Eye_L', 'External', 'Esophagus', 'CTV_Mid01', 'CTV_Mid00', 'CTV_Low', 'CTV_High', 'Cochlea_R', 'Cochlea_L', 'Brainstem', 'Brain', 'Bone', 'A_Carotid_R', 'A_Carotid_L', 'Dose 75[%]']
Visualizing contours in 2D and 3D...
Found 196 unique Z positions
Done
